In [ ]:
from evaluate import load
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
os.environ['MPLCONFIGDIR'] = '/nfs/nas-6.1/wclu/cache'

In [ ]:
result = []
directory = '/nfs/nas-6.1/wclu/prompt_compression/output'
for root, dirs, files in os.walk(directory):
    for filename in tqdm(files):
        if 'random' in filename:
            df = pd.read_json(os.path.join(root, filename))
            config = filename.replace('random_','').replace('.json','').split('_')
            config.append(df)
            result.append(config)
df = pd.DataFrame(result, columns=['size','c_rate', 'df'])

In [ ]:
bertscore = load("bertscore")
def evaluate_bert(df):
    seq_predictions = df['seq_com_output'].tolist()
    para_predictions = df['para_com_output'].tolist()
    references = df['output'].tolist()
    seq_score = bertscore.compute(predictions=seq_predictions, references=references, model_type="microsoft/deberta-large-mnli", batch_size=16)
    para_score = bertscore.compute(predictions=para_predictions, references=references, model_type="microsoft/deberta-large-mnli", batch_size=16)
    seq_f1 = np.mean(seq_score['f1'])
    para_f1 = np.mean(para_score['f1'])
    return format(seq_f1,'.4f'), format(para_f1,'.4f')


In [ ]:
seq_bertscore = []
para_bertscore = []
for i in tqdm(range(len(df))):
    bert_score = evaluate_bert(df['df'][i])
    seq_bertscore.append(bert_score[0])
    para_bertscore.append(bert_score[1])

In [ ]:
df['seq_bertscore'] = seq_bertscore
df['para_bertscore'] = para_bertscore

In [ ]:
perplexity = load("perplexity",module_type="metric")
def evaluate_perplexity(df, size):
    #references = ('User: '+df['prompt']+'\nAssistant: '+df['output']).tolist()
    seq_predictions = (df['prompt']+' '+df['seq_com_output']).tolist()
    para_predictions = (df['prompt']+' '+df['para_com_output']).tolist()
    #ref_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=references, batch_size=16)
    seq_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=seq_predictions, batch_size=32)
    para_ppl = perplexity.compute(model_id=f'meta-llama/Llama-2-{size}-chat-hf', predictions=para_predictions, batch_size=32)
    #return format(ref_ppl['mean_perplexity'],'.3f'), format(seq_ppl['mean_perplexity'],'.3f'), format(para_ppl['mean_perplexity'],'.3f')
    return format(seq_ppl['mean_perplexity'],'.3f'), format(para_ppl['mean_perplexity'],'.3f')
    


In [ ]:
seq_perplexity = []
para_perplexity = []
for i in tqdm(range(len(df))):
    ppl = evaluate_perplexity(df['df'][i], df['size'][i])
    seq_perplexity.append(ppl[0])
    para_perplexity.append(ppl[1])

In [ ]:
df['seq_ppl'] = seq_perplexity
df['para_ppl'] = para_perplexity

In [ ]:
df[df['size']=='13b'].sort_values('c_rate')

In [ ]:
df[df['size']=='7b'].sort_values('c_rate')

## GPT

In [ ]:
references = ('A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user\'s questions.\nUSER: ' + df['df'][11]['prompt']+'\nASSISTANT: '+df['df'][11]['seq_com_output']).tolist()
#references = ('USER: ' + df['df'][11]['prompt']+'\nASSISTANT: '+df['df'][11]['seq_com_output']).tolist()
print(references[0])

In [ ]:
ref_ppl = perplexity.compute(model_id=f'lmsys/vicuna-7b-v1.5', predictions=references, batch_size=32)
print(format(ref_ppl['mean_perplexity'],'.3f'))


In [ ]:
score = []

In [ ]:
for i in tqdm(range(len(score),5)):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {
            "role": "system",
            "content": "Imagine that you have a super-intelligent AI assistant, and that you require help with the following question. Which answer best satisfies your needs?"
            },
            {
            "role": "user",
            "content": f"Question: {prompt_list[i]}\n\nAnswer G: <{references[i]}>\n\nAnswer F: <{seq_predictions[i]}>\n\nComparing these two answers, which answer is better?\n◼ Answer F is significantly better.\n◼ Answer U is significantly better.\n◼ Neither is significantly better."
            }
        ],
        temperature=0,
        max_tokens=30,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    score.append(response.choices[0].message.content)